In [13]:
import pandas as pd 
df_model = pd.read_csv("final_learning_potential_dataset.csv")
print(df_model.shape)
df_model.head()

(427, 7)


,stress_congeneric_score,control_congeneric,self_efficacy_congeneric,depression_congeneric_score,somatic_congeneric,cog_affect_congeneric,learning_potential_score
0,-0.994499,-0.568381,-1.568341,-0.852725,1.522905,-0.294627,2.153331
1,-1.601830,-1.258011,-1.846971,-1.430514,1.403349,-1.390766,3.369578
2,0.183967,0.516081,-0.397377,-0.347319,-0.019725,-0.594503,0.234547
3,0.419527,0.581236,0.063644,-0.825759,1.003940,-0.594503,0.534051
4,-1.093718,-1.413048,-0.214986,0.870991,-1.372580,0.501635,-0.020603


In [11]:
#kurulum 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

REQ_COLS = [
    "stress_congeneric_score",
    "control_congeneric",
    "self_efficacy_congeneric",
    "depression_congeneric_score",
    "somatic_congeneric",
    "cog_affect_congeneric",
    "learning_potential_score",
]

df = pd.read_csv("final_learning_potential_dataset.csv")
assert set(REQ_COLS).issubset(df.columns), "Beklenen kolonlar eksik!"
Xcols = [c for c in REQ_COLS if c != "learning_potential_score"]
X = df[Xcols].copy()
y = df["learning_potential_score"].copy()
print(df.shape, df[Xcols].isna().sum().sum(), df["learning_potential_score"].isna().sum())


(427, 7) 0 0


In [12]:
#Özelliklerin gerçekten z-skor (standart) olup olmadığı
desc = X.agg(["mean","std"]).T.round(3)
display(desc)


,mean,std
stress_congeneric_score,-0.0,1.065
control_congeneric,0.0,1.084
self_efficacy_congeneric,0.0,1.117
depression_congeneric_score,-0.0,1.048
somatic_congeneric,-0.0,1.089
cog_affect_congeneric,-0.0,1.069


In [7]:
#kayıtlı pca skoru ile karşılaştırma 
scaler = StandardScaler()
Z = scaler.fit_transform(X)

pca = PCA(n_components=1, random_state=0)
pc1_new = pca.fit_transform(Z).ravel()   # yeninden hesaplanan PC1

# 2a) Kayıtlı skorla korelasyon
r = np.corrcoef(y, pc1_new)[0,1]
print("corr(y, pc1_new) =", r)

# 2b) Ölçeksiz hata (ölçek farkı yoksa ~0 olmalı)
mse = np.mean(( (y / np.std(y)) - (pc1_new / np.std(pc1_new)) )**2)
print("scale-free MSE ~", mse)


corr(y, pc1_new) = -0.9999999999999998
scale-free MSE ~ 4.000000000000001


In [16]:
#learning potantial sütunu işaretleri mantıksal açıdan verimli mi? 
loadings = pd.Series(pca.components_[0], index=X.columns).sort_values(ascending=False)
print("PC1 yükleri:")
display(loadings)

expected = {
    "control_congeneric": +1,
    "self_efficacy_congeneric": +1,
    "stress_congeneric_score": -1,
    "depression_congeneric_score": -1,
    "somatic_congeneric": -1,
    "cog_affect_congeneric": -1,
}

corrs = df.loc[:, list(X.columns) + ["learning_potential_score"]].corr()["learning_potential_score"].drop("learning_potential_score").round(3)
print("learning_potential_score ile korelasyonlar:")
display(corrs.sort_values(ascending=False))


PC1 yükleri:


depression_congeneric_score    0.432949
stress_congeneric_score        0.428217
cog_affect_congeneric          0.417188
control_congeneric             0.401753
self_efficacy_congeneric       0.369307
somatic_congeneric            -0.396670
dtype: float64

learning_potential_score ile korelasyonlar:


somatic_congeneric             0.855
self_efficacy_congeneric      -0.796
control_congeneric            -0.866
cog_affect_congeneric         -0.899
stress_congeneric_score       -0.923
depression_congeneric_score   -0.933
Name: learning_potential_score, dtype: float64

In [9]:
reverse_cols = [
    "stress_congeneric_score",
    "depression_congeneric_score",
    "somatic_congeneric",
    "cog_affect_congeneric",
]

for col in reverse_cols:
    mean_val = df[col].mean()
    if mean_val > 0:
        df[col] = -df[col]
        print(f"{col} yönü düzeltildi (ortalama {round(mean_val,3)})")
    else:
        print(f"{col} zaten doğru yönlü (ortalama {round(mean_val,3)})")

df.to_csv("final_learning_potential_dataset.csv", index=False)
print("\n✅ Yönsel düzeltme tamamlandı ve dosya güncellendi.")


stress_congeneric_score zaten doğru yönlü (ortalama -0.0)
depression_congeneric_score zaten doğru yönlü (ortalama -0.0)
somatic_congeneric zaten doğru yönlü (ortalama -0.0)
cog_affect_congeneric zaten doğru yönlü (ortalama -0.0)

✅ Yönsel düzeltme tamamlandı ve dosya güncellendi.


In [15]:
df.mean().round(3)


stress_congeneric_score       -0.0
control_congeneric             0.0
self_efficacy_congeneric       0.0
depression_congeneric_score   -0.0
somatic_congeneric            -0.0
cog_affect_congeneric         -0.0
learning_potential_score       0.0
dtype: float64

In [ ]:
# pca sonrası kalıcı şekilde yön düzeltme 
import numpy as np
import pandas as pd 
 
df = pd.read_csv("final_learning_potential_dataset.csv")

feature_cols = [
    "stress_congeneric_score",
    "control_congeneric",
    "self_efficacy_congeneric",
    "depression_congeneric_score",
    "somatic_congeneric",
    "cog_affect_congeneric",
]

expected_sign = {
    "control_congeneric": +1,
    "self_efficacy_congeneric": +1,
    "stress_congeneric_score": -1,
    "depression_congeneric_score": -1,
    "somatic_congeneric": -1,
    "cog_affect_congeneric": -1,
}

assert "learning_potential_score" in df.columns, "learning_potential_score sütunu yok!"

corrs = df[feature_cols + ["learning_potential_score"]].corr()["learning_potential_score"].drop("learning_potential_score")
print("Önceki korelasyon işaretleri (learning_potential_score ile):\n")
print(corrs.sort_values(ascending=False).round(3), "\n")

changed = []
for col in feature_cols:
    sign_now = np.sign(corrs[col]) if not np.isnan(corrs[col]) else 0
    want = expected_sign[col]
    # Eğer işaret beklenenle uyuşmuyorsa, kolonu tersine çevir
    if sign_now != 0 and sign_now != want:
        df[col] = -df[col]
        changed.append(col)



if changed:
    print("🔄 Ters çevrilen sütunlar:", changed)
else:
    print("✅ Tüm sütunlar beklenen yönle uyumlu görünüyordu; değişiklik yapılmadı.")


corrs_new = df[feature_cols + ["learning_potential_score"]].corr()["learning_potential_score"].drop("learning_potential_score")
print("\nDüzeltme sonrası korelasyon işaretleri:\n")
print(corrs_new.sort_values(ascending=False).round(3))

Önceki korelasyon işaretleri (learning_potential_score ile):

somatic_congeneric             0.855
self_efficacy_congeneric      -0.796
control_congeneric            -0.866
cog_affect_congeneric         -0.899
stress_congeneric_score       -0.923
depression_congeneric_score   -0.933
Name: learning_potential_score, dtype: float64 

🔄 Ters çevrilen sütunlar: ['control_congeneric', 'self_efficacy_congeneric', 'somatic_congeneric']

Düzeltme sonrası korelasyon işaretleri:

control_congeneric             0.866
self_efficacy_congeneric       0.796
somatic_congeneric            -0.855
cog_affect_congeneric         -0.899
stress_congeneric_score       -0.923
depression_congeneric_score   -0.933
Name: learning_potential_score, dtype: float64


In [19]:
#yeniden pca fit uygulaması 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

# 1️⃣ PCA için sadece özellik sütunlarını alalım
X = df[feature_cols].copy()

# 2️⃣ Standardize et (z-skor)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3️⃣ PCA'yı 1 bileşenle fit et
pca = PCA(n_components=1, random_state=0)
pc1 = pca.fit_transform(X_scaled).ravel()

# 4️⃣ Ekseni domain’e göre yönlendir (pozitif kontrol/self_efficacy ile hizala)
ref_corr = np.corrcoef(pc1, df["control_congeneric"])[0,1] + np.corrcoef(pc1, df["self_efficacy_congeneric"])[0,1]
if ref_corr < 0:
    pc1 = -pc1
    print("🔄 PCA ekseni domain mantığına göre ters çevrildi.")

# 5️⃣ Yeni PCA skorunu dataframe’e ekle
df["learning_potential_score"] = pc1

# 6️⃣ Dosyayı kaydet
df.to_csv("final_learning_potential_dataset.csv", index=False)
print("✅ PCA yeniden fit edildi ve yönü sabitlendi. Güncellenmiş CSV kaydedildi.")

# 7️⃣ Son kontrol
corrs_final = df[feature_cols + ["learning_potential_score"]].corr()["learning_potential_score"].drop("learning_potential_score")
print("\n📊 Nihai korelasyonlar (learning_potential_score ile):")
print(corrs_final.sort_values(ascending=False).round(3))


🔄 PCA ekseni domain mantığına göre ters çevrildi.
✅ PCA yeniden fit edildi ve yönü sabitlendi. Güncellenmiş CSV kaydedildi.

📊 Nihai korelasyonlar (learning_potential_score ile):
control_congeneric             0.866
self_efficacy_congeneric       0.796
somatic_congeneric            -0.855
cog_affect_congeneric         -0.899
stress_congeneric_score       -0.923
depression_congeneric_score   -0.933
Name: learning_potential_score, dtype: float64


In [20]:
# NaN / eksik değer kontrolü
print("Eksik değer sayıları:\n", df.isna().sum())
print("\nToplam eksik değer:", df.isna().sum().sum())

# Temel istatistiksel özet
print("\nVeri seti özet istatistikleri:")
display(df.describe().T.round(3))


Eksik değer sayıları:
 stress_congeneric_score        0
control_congeneric             0
self_efficacy_congeneric       0
depression_congeneric_score    0
somatic_congeneric             0
cog_affect_congeneric          0
learning_potential_score       0
dtype: int64

Toplam eksik değer: 0

Veri seti özet istatistikleri:


,count,mean,std,min,25%,50%,75%,max
stress_congeneric_score,427.0,-0.0,1.065,-2.928,-0.792,0.088,0.780,2.940
control_congeneric,427.0,-0.0,1.084,-2.607,-0.783,-0.053,0.782,2.715
self_efficacy_congeneric,427.0,-0.0,1.117,-2.963,-0.774,-0.064,0.803,2.836
depression_congeneric_score,427.0,-0.0,1.048,-1.555,-0.882,-0.124,0.795,2.490
somatic_congeneric,427.0,0.0,1.089,-1.667,-0.976,-0.114,0.740,2.335
cog_affect_congeneric,427.0,-0.0,1.069,-1.391,-0.819,-0.295,0.771,2.392
learning_potential_score,427.0,0.0,2.158,-5.947,-1.495,-0.019,1.748,4.280


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("final_learning_potential_dataset.csv")

cols = [
    "stress_congeneric_score",
    "control_congeneric",
    "self_efficacy_congeneric",
    "depression_congeneric_score",
    "somatic_congeneric",
    "cog_affect_congeneric",
]

data = df[cols]

# Cronbach's alpha hesabı
item_variances = data.var(axis=0, ddof=1)
total_var = data.sum(axis=1).var(ddof=1)
n_items = data.shape[1]
alpha = (n_items / (n_items - 1)) * (1 - (item_variances.sum() / total_var))

print(f"Cronbach's Alpha: {alpha:.3f}")


Cronbach's Alpha: -0.115


In [3]:
corrs = df[[
    "stress_congeneric_score",
    "control_congeneric",
    "self_efficacy_congeneric",
    "depression_congeneric_score",
    "somatic_congeneric",
    "cog_affect_congeneric"
]].corr().round(2)

print(corrs)


                             stress_congeneric_score  control_congeneric  \
stress_congeneric_score                         1.00               -0.96   
control_congeneric                             -0.96                1.00   
self_efficacy_congeneric                       -0.83                0.64   
depression_congeneric_score                     0.73               -0.68   
somatic_congeneric                              0.65               -0.62   
cog_affect_congeneric                           0.72               -0.67   

                             self_efficacy_congeneric  \
stress_congeneric_score                         -0.83   
control_congeneric                               0.64   
self_efficacy_congeneric                         1.00   
depression_congeneric_score                     -0.62   
somatic_congeneric                              -0.53   
cog_affect_congeneric                           -0.62   

                             depression_congeneric_score  somatic_c

In [4]:
cols_pos = ["control_congeneric", "self_efficacy_congeneric"]
cols_neg = ["stress_congeneric_score", "depression_congeneric_score", "somatic_congeneric", "cog_affect_congeneric"]

def cronbach_alpha(df_subset):
    item_var = df_subset.var(axis=0, ddof=1)
    total_var = df_subset.sum(axis=1).var(ddof=1)
    n = df_subset.shape[1]
    return (n / (n - 1)) * (1 - (item_var.sum() / total_var))

print("Pozitif grup alpha:", cronbach_alpha(df[cols_pos]))
print("Negatif grup alpha:", cronbach_alpha(df[cols_neg]))


Pozitif grup alpha: 0.7776142081797659
Negatif grup alpha: 0.9375067687587679
